In [ ]:
using Revise

using Pkg
Pkg.activate("..")

import PolynomialQTT
import QuanticsGrids as QG
import TensorCrossInterpolation as TCI
using PythonPlot

using PythonCall: PythonCall
using PythonPlot: pyplot as plt, Figure

# Displays the matplotlib figure object `fig` and avoids duplicate plots.
_display(fig::Figure) = isinteractive() ? (fig; plt.show(); nothing) : Base.display(fig)
_display(fig::PythonCall.Py) = _display(Figure(fig))

R = 20
N = 3
a, b = (0.0, 0.0, 0.0), (1.0, 1.0, 1.0)

fr(r) = r == 0.0 ? 0.0 : 1.0 / r
f(x, y, z) = fr(sqrt(x^2 + y^2 + z^2))

K = 10
tt = PolynomialQTT.interpolatemultiscale(f, a, b, R, K, [(0.0, 0.0, 0.0)]; tolerance=1e-6)
maximum(TCI.linkdims(tt))

In [ ]:
plotgrid = 2:2^(R-8):2^R

grid = QG.DiscretizedGrid{N}(R, a, b)
quanticsinds = [QG.grididx_to_quantics(grid, (i, i, i)) for i in plotgrid]
xs = [QG.grididx_to_origcoord(grid, (i, i, i)) for i in plotgrid]
origdata = [f(x...) for x in xs]
ttdata = tt.(quanticsinds)

fig, ax = plt.subplots()
ax.semilogy(first.(xs), origdata, label="Original", marker="x")
ax.semilogy(first.(xs), ttdata, label="Interpolation", marker="+")
ax.semilogy(first.(xs), origdata .- ttdata, label="Error", marker="x")
ax.legend()
ax.set_xlabel("r")
ax.set_title("3D Coulomb kernerl bonddim=$(maximum(TCI.linkdims(tt)))")
plt.savefig("coulombkernel3d.pdf")
_display(fig)